* regression:
  * check R^2 between percentage changes to control for spurious trends

##### risk associated with fixed income instruments
   * interest rate risk

      the risk that change in interest rate affects the demand for the fixed income of a bond, which in turn affects the bond price and yield

      interest rate risk is the most predominant risk for fixed income instrument

   * credit risk

      the risk that the issuer cannot pay back the principal, i.e. defaulting

   * inflation risk

      the risk that the inflation will erode the cash flow generated from fixed income instruments

   * financing/liquidity risk

      lack of liquidity (ability to buy and sell without impacting market price) could be caused by lack of financing market (i.e. inability to borrow) and can cause uncorrelated assets move together as there is need to sell to raise fund

   * tax risk

      fixed income cash flows are taxable

   * regulatory risk

##### zero-coupon bond
a bond that pays no interest in the interim and instead pays a fixed amount at maturity

##### present value of cash flow
what is the value such that it can generate the cash flow given market rate r?
e.g. zero coupon bond that matures in 10 years with $100 principal
the principal = pv * (1 + y)^10

##### pv of fixed coupon bond
a fixed coupon bond can be thought of as a series zero-coupon bonds that matures semi-anually, and a zero-coupon bond with the principal
which would generate the same cash flow as the fixed coupon bond
the pv of a fixed coupon bond is therefore the sum of pv's of these zero-coupon bonds

##### yield to maturity
if the market price differs from the calculated pv, then the "effective" internal yield y is no longer the same as the market interest rate
we call this effective yield "yield to maturity" or simply yield
the price of the bond is inversely proportional to yield

##### yield curve
a plot of yield vs bond maturities
the yield curve changes through time as market condition changes
so really it would be interesting to see the plot in three axis: yield, maturities, time
yield curve with positive slope may suggest the investors demand better yield
for longer maturities, as they may speculate the interest rate may rise in the future

##### Leveraging
A hedge fund may gain larger PnL by leveraging i.e. by borrowing money to fund a trade.

e.g. a 10:1 leverage

If I have $100, and the bond costs $100. I could borrow $900 and buy 10 issuance of the bond.
If the bond price increases or decreases by $2, instead of a 2% PnL, leveraging could result in 20% PnL

##### Repo Market
Repo Market is the market for short-tearm loans, e.g. over-night loans

In a repo trade, a lender such as a bank or a mutual fund, will lend out cash with the agreement to receive it tat a later date.
The borrower of the cash, such as hedge fund, will lend out bonds as collateral with the agreement to repurchase them later at a fixed price.

e.g. bank lend out $98 cash, in exchange for $100 worth of bonds, the hedge fund buys back the bond at $99,
the 1 dollar cost is the repo rate

For most Treasuries, the repo rate is near a standard rate known as the general collateral rate, GC rate for short.
The GC rate is set by the repo market participants, based on the supply and demand of short-term funding, and the collateral.

If the collateral of a specific bond is in high demand, then it maybe repo'd at a lower rate than the GC rate known as the special rate. This tends to be the case for some on-the-run bonds or cheapest-to-deliver bonds in a bond futures basket.

##### Carry
Carry is the net income from owning an asset, in the context of bonds, carry is the difference between the coupon and the financing cost of the position, e.g. repo rates

In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

from bond import FixedCouponBond

#### Present Value vs Yield Plot

In [87]:
from plotly.subplots import make_subplots

y = 0.05
p = 100
c = 5

fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("linear for small dy", "non-linear for big dy"),
    x_title="Yield",
    y_title="Present Value",
    )

yields = [i / 100 for i in range(3, 8)]
pvs = [FixedCouponBond(c, p, maturity=5).present_value(y) for y in yields]
fig = fig.add_trace(go.Scatter(x=yields, y=pvs, name="m = 5"), row=1, col=1)
pvs = [FixedCouponBond(c, p, maturity=10).present_value(y) for y in yields]
fig = fig.add_trace(go.Scatter(x=yields, y=pvs, name="m = 10"), row=1, col=1)

yields = [i / 100 for i in range(1, 100)]
pvs = [FixedCouponBond(c, p, maturity=5).present_value(y) for y in yields]
fig = fig.add_trace(go.Scatter(x=yields, y=pvs, name="m = 5"), row=1, col=2)
pvs = [FixedCouponBond(c, p, maturity=10).present_value(y) for y in yields]
fig = fig.add_trace(go.Scatter(x=yields, y=pvs, name="m = 10"), row=1, col=2)

fig.show()


* effect of changing maturity

    Notice that the maturity increases the slope, meaning the PV is more sensitive to change in yield
    this is because money has timed value associated with it

* if the market interest rate is higher than the coupon

    then the bond has to be discounted to offset the lower coupon, and higher
    maturity means the investors are taking on this risk for longer duration
    so the bond has to be further discounted 

* if the market interest rate is lower than the coupon

    then the bond is sold at a premium, as we are beating the market interest
    rate, higher maturity means longer period with higher returns, therefore
    results in higher premium
* if the interest rate is the same as the coupon

    The bond must be sold equal to the value of the principal, as the fixed income cashflow matches exactly with the market interest rate


#### Non-linear relationship between PV and yield
Notice under small yield changes, the relationship appears to be linear, this is not the case for big yield changes, this means we need higher order derivatives in order to approximate how the PV changes under big yield changes, the second order derivative is called convexity

The convexity is more important in more volatile market conditions, as it has larger yield changes

In [55]:
y = 0.05
c = 5
p = 100
m = 5

bond = FixedCouponBond(c, p, m)
pv = bond.present_value(y)

# the PV equal to the principal when coupon equal to the market interest rate
print("pv", pv)
print("total_raw_return", sum(cash for cash, _ in bond.cash_flow))
print("macaulay_duration", bond.macaulay_duration(y))
print("modified duration", bond.modified_duration(y))
print("cash_flow", bond.cash_flow)

pv 100.00000000000009
total_raw_return 125.0
macaulay_duration 4.485432764622604
modified duration 4.376031965485468
cash_flow [(2.5, 0.5), (2.5, 1.0), (2.5, 1.5), (2.5, 2.0), (2.5, 2.5), (2.5, 3.0), (2.5, 3.5), (2.5, 4.0), (2.5, 4.5), (2.5, 5.0), (100, 5)]


#### Macaulay and Modified Macaulay Duration
Macaulay duration is essentially the first derivative of the present value wrt interest rate

There are two versions, modified Macaulay duration and Macaulay duration both has unit of time
* MD measures the weighted average time to receive the bond's cash flows, taking into account the size and timing of those cash flows
* Modified MD is the MD adjusted so it is the first derivative of PV

    The modified MD used as a first order approximation of how much the PV changes under small perturbation of interest rate y



$$Modified MD = -\frac{1}{PV} \frac{\partial PV}{\partial y}$$
$$Modified MD = \frac{MD}{1 + \frac{y}{n}}$$
where n is the coupon frequency

In [46]:
c = 5
p = 100
m = 5

bond = FixedCouponBond(c, p, m)
# if the original interest rate is 0.05 and it changes to 0.06
# then we expect the present value to decrease by the modified duration

print(f"""
{bond.present_value(0.06) - bond.present_value(0.05)=}
{-bond.modified_duration(0.05)=}

Notice under larger change in interest rate, the modified duration no longer a good approximate.
{bond.present_value(0.20) - bond.present_value(0.05)=}
{-bond.modified_duration(0.05) * 15=}
Therefore The modified duration is a good approximate of how much PV changes under small changes of interest rate.

Notice how the modified duration changes when yield changes, this is because
there is a non-linear relationship between the PV and the yield.
So the first derivative i.e. duration depends on yield as well.

{bond.modified_duration(0.05)=}
{bond.modified_duration(0.06)=}

In order to better approximate how the PV changes under larger changes in the interest rate, we need to calculate higher order derivatives
The second order derivative is called the Convexity

""")


bond.present_value(0.06) - bond.present_value(0.05)=-4.265101418388028
-bond.modified_duration(0.05)=-4.376031965485468

Notice under larger change in interest rate, the modified duration no longer a good approximate.
bond.present_value(0.20) - bond.present_value(0.05)=-46.08425329278524
-bond.modified_duration(0.05) * 15=-65.64047948228202
Therefore The modified duration is a good approximate of how much PV changes under small changes of interest rate.

Notice how the modified duration changes when yield changes, this is because
there is a non-linear relationship between the PV and the yield.
So the first derivative i.e. duration depends on yield as well.

bond.modified_duration(0.05)=4.376031965485468
bond.modified_duration(0.06)=4.34143554134103

In order to better approximate how the PV changes under larger changes in the interest rate, we need to calculate higher order derivatives
The second order derivative is called the Convexity




In [67]:
r = 0.05
t = 10
n = 100000
discrete = (1 + r / n) **(n*t)
continuous = np.exp(r * t)
100 * (continuous - discrete) / discrete
discrete

1.6487210647252784